In [ ]:
--top 3 products by sales, top 3 employee by salary, within cat/dept
--top2 highest salried employee
select * from emp
order by salary desc
limit 2

--top2 highest salried employee on each dept
select * from
(
    SELECT *,
    row_number() over(PARTITION BY department_id ORDER BY salary desc) as rnk 
    from emp
) a
where rnk <= 2

In [ ]:
--2. Top 5 products by sales
with cte as
(
    SELECT product_id, sum(sales) as sales
    from orders
    GROUP BY product_id
)
select * from cte 
order by sales desc
limit 5

--2.  Top 5 products by sales in each category
with cte as
(
    SELECT category,product_id, sum(sales) as sales
    from orders
    GROUP BY category,product_id
)
select * from
(
select *,
row_number() over(PARTITION BY category order by sales desc) as rnk
from cte 
) a
where rnk <= 5

In [ ]:
--2. yoy growth/product with sales more than previous year sales
with cte as
(
    select
    extract(year from order_date) as year_order, sum(sales) as sales
    from orders
    GROUP BY extract(year from order_date)
),
cte2 as
(
    select *,
    lag(sales,1,sales) over(order by year_order) as previous_year_sales
    from cte
)
select *, 
(sales - previous_year_sales)*100/previous_year_sales as yoy
from cte2 


--yoy growth/product with sales more than previous year sales in each category
with cte as
(
    select
    category, extract(year from order_date) as year_order, sum(sales) as sales
    from orders
    GROUP BY category,extract(year from order_date)
),
cte2 as
(
    select *,
    lag(sales,1,sales) over(PARTITION BY category order by year_order) as previous_year_sales
    from cte
)
select *, 
(sales - previous_year_sales)*100/previous_year_sales as yoy
from cte2

In [ ]:
---3-- running/cumulative sales year wise / rolling n months sales

with cte as
(
    select extract(year from order_date) as year_order, sum(sales) as sales
    from orders
    GROUP BY extract(year from order_date)
)
select *, 
sum(sales) over(order by year_order) as cumulative_sum
from cte

--for category wise
with cte as
(
    select category,extract(year from order_date) as year_order, sum(sales) as sales
    from orders
    GROUP BY extract(year from order_date)
)
select *, 
sum(sales) over(PARTITION BY category order by year_order) as cumulative_sum
from cte


--rolling 3 months sales, previous 2 month + current month
with cte as
(
    select category,extract(year from order_date) as year_order, extract(month form order_date) as month_order_date, sum(sales) as sales
    from orders
    GROUP BY extract(year from order_date), extract(month form order_date)
)
select *, 
sum(sales) over(order by year_order, month_order_date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as cumulative_sum
from cte

with cte as
(
    select category,extract(year from order_date) as year_order, extract(month form order_date) as month_order_date, sum(sales) as sales
    from orders
    GROUP BY extract(year from order_date), extract(month form order_date)
)
select *, 
sum(sales) over(order by year_order, month_order_date ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING) as cumulative_sum
from cte

In [ ]:
--4. pivoting. convert rows into columns year wise each category sales

select extract(year from order_date) as year_order,
sum(case when category = 'Furniture' then sales else 0 end) as furn_sales,
sum(case when category = 'Office Supplies' then sales else 0 end) as os_sales,
sum(case when category = 'Technology' then sales else 0 end) as tech_sales
from orders
GROUP BY  extract(year from order_date)
